In [20]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_community.embeddings import HuggingFaceEmbeddings  # Changed from OpenAIEmbeddings
from langchain.vectorstores import Chroma
import shutil
from langchain.schema import Document
import sys
from langchain_community.document_loaders import TextLoader

pth = r"C:\Users\Devesh\Desktop\genai\alice_in_wonderland.md"

In [21]:
def load_documents():
    # Load documents from a directory
    loader = TextLoader(file_path="C:/Users/Devesh/Desktop/genai/alice_in_wonderland.md", encoding="utf-8")
    documents = loader.load()
    return documents

In [22]:
def generatedatastore():
    documents=load_documents()
    chunks=textsplitter(documents)
    savechroma(chunks)

In [23]:
def textsplitter(document = list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
        add_start_index=False,
    )
    chunks=text_splitter.split_documents(document)
    document =chunks[10]
    return chunks

chroma is used for database

In [24]:
def savechroma(chunks: list[Document]):
    if os.path.exists("chromaopth"):
        shutil.rmtree("chromaopth")
    
    # Using HuggingFace embeddings instead of OpenAI
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    db = Chroma.from_documents(
        chunks, embeddings, persist_directory="chromaopth"
    )
    db.persist()
    

In [25]:
if __name__ == "__main__":
    generatedatastore()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Devesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Devesh\AppData\Local\Temp\ipykernel_34440\3212476951.py", line 2, in <module>
    generatedatastore()
  File "C:\Users\Devesh\AppData\Local\Temp\ipykernel_34440\4014463615.py", line 4, in generatedatastore
    savechroma(chunks)
  File "C:\Users\Devesh\AppData\Local\Temp\ipykernel_34440\3169525408.py", line 7, in savechroma
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Devesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py", line 224, in warn_if_direct_instance
    return wrapped(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Devesh\AppData\Local\Programs\Python\Python311\

In [28]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import shutil

# Configuration
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
MODEL_NAME = "all-MiniLM-L6-v2"  # Lightweight model

def load_documents(file_path):
    loader = TextLoader(file_path, encoding="utf-8")
    return loader.load()

def split_documents(documents):
    return RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len
    ).split_documents(documents)

def create_vector_store(chunks, persist_dir="chroma_db"):
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
    
    embeddings = HuggingFaceEmbeddings(
        model_name=MODEL_NAME,
        model_kwargs={"device": "cpu"}  # Force CPU usage
    )
    
    db = Chroma.from_documents(
        chunks,
        embeddings,
        persist_directory=persist_dir
    )
    db.persist()
    return db

if __name__ == "__main__":
    try:
        print("Loading documents...")
        docs = load_documents("C:/Users/Devesh/Desktop/genai/alice_in_wonderland.md")
        
        print("Splitting documents...")
        chunks = split_documents(docs)
        
        print("Creating vector store...")
        db = create_vector_store(chunks)
        
        print("Successfully created vector store!")
    except Exception as e:
        print(f"Error: {str(e)}")
        print("Please ensure all dependencies are properly installed")

Loading documents...
Splitting documents...
Creating vector store...
Error: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
Please ensure all dependencies are properly installed


In [29]:
import os
import shutil
import numpy as np
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

def verify_environment():
    """Check critical package versions"""
    try:
        assert np.__version__ == '1.23.5', f"NumPy version {np.__version__} incompatible"
        import torch
        assert torch.__version__.startswith('2.0.1'), f"Torch version {torch.__version__} incompatible"
        return True
    except Exception as e:
        print(f"Environment verification failed: {str(e)}")
        return False

def load_and_process(file_path):
    if not verify_environment():
        raise EnvironmentError("Package version mismatch detected")
    
    try:
        print("Loading documents...")
        loader = TextLoader(file_path, encoding='utf-8')
        documents = loader.load()
        
        print("Splitting documents...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            length_function=len
        )
        chunks = text_splitter.split_documents(documents)
        
        print("Initializing embeddings...")
        embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        
        print("Creating vector store...")
        if os.path.exists("chroma_db"):
            shutil.rmtree("chroma_db")
            
        db = Chroma.from_documents(
            chunks,
            embeddings,
            persist_directory="chroma_db"
        )
        db.persist()
        print("Vector store created successfully!")
        return db
        
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        load_and_process("C:/Users/Devesh/Desktop/genai/alice_in_wonderland.md")
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        print("Recommended solution: Create fresh virtual environment")
        print("Run: python -m venv new_env && new_env\\Scripts\\activate")
        print("Then reinstall packages with exact versions")

Environment verification failed: NumPy version 2.3.0 incompatible
Fatal error: Package version mismatch detected
Recommended solution: Create fresh virtual environment
Run: python -m venv new_env && new_env\Scripts\activate
Then reinstall packages with exact versions
